# User type example: `ModInt`

Sometimes you want a new kind of thing. In Julia types are just some data which we can add new behaviors to. New behavior is added by defining methods for existing and new generic functions. In this example, we're going to define a modular integer type called `ModInt`, which does "clock arithmetic" – with some number of "hours".

## Defining a new type

In [12]:
struct ModInt{n} <: Integer
    k::Int
    (ModInt{n}(k) where {n}) = new(mod(k,n))
end

In [13]:
a = ModInt{13}(213)

ModInt{13}(5)

In [14]:
b = ModInt{13}(987)

ModInt{13}(12)

In [15]:
dump(a)

ModInt{13}
  k: Int64 5


In [16]:
213 % 13

5

In [11]:
987 % 13

12

That's it – that's all there is to creating a new type. Of course, it can't do much yet...

In [17]:
a + b

LoadError: [91m+ not defined for ModInt{13}[39m

## Customizing behavior

First, some nicer display:

In [18]:
Base.show{n}(io::IO, k::ModInt{n}) = print(io, "$(k.k) mod $n")

In [19]:
a

5 mod 13

In [20]:
b

12 mod 13

In [21]:
import Base: +, -, *, /, inv

In [22]:
# basic arithmetic

+(a::ModInt{n}, b::ModInt{n}) where {n} = ModInt{n}(a.k + b.k)
-(a::ModInt{n}, b::ModInt{n}) where {n} = ModInt{n}(a.k - b.k)
*(a::ModInt{n}, b::ModInt{n}) where {n} = ModInt{n}(a.k * b.k)
-(a::ModInt{n}) where {n} = ModInt{n}(-a.k)

/(a::ModInt{n}, b::ModInt{n}) where {n} = a*inv(b)
inv(a::ModInt{n}) where {n} = ModInt{n}(invmod(a.k, n))

# interacting with regular integers

Base.promote_rule(::Type{ModInt{n}}, ::Type{Int}) where {n} = ModInt{n}
Base.convert(::Type{ModInt{n}}, i::Int) where {n} = ModInt{n}(i)

In [23]:
a + b

4 mod 13

In [24]:
a*b

8 mod 13

In [26]:
(a/b)*b

5 mod 13

In [28]:
inv(b)

12 mod 13

In [34]:
inv(ModInt{15}(8))

2 mod 15

With just a bit of arithmetic, we can do quite a bit:

In [35]:
A = map(ModInt{13}, rand(Int, 10, 10))

10×10 Array{ModInt{13},2}:
  5 mod 13   0 mod 13  10 mod 13  …  11 mod 13   4 mod 13   1 mod 13
  5 mod 13   4 mod 13   1 mod 13      5 mod 13   0 mod 13  12 mod 13
  3 mod 13   5 mod 13   6 mod 13      6 mod 13   1 mod 13   6 mod 13
  3 mod 13   6 mod 13   8 mod 13      8 mod 13  12 mod 13  12 mod 13
  9 mod 13  12 mod 13   9 mod 13      9 mod 13   9 mod 13   3 mod 13
 11 mod 13   2 mod 13   5 mod 13  …  11 mod 13   8 mod 13   2 mod 13
  3 mod 13   1 mod 13  12 mod 13      5 mod 13   5 mod 13   4 mod 13
 12 mod 13   2 mod 13   9 mod 13      3 mod 13   6 mod 13   2 mod 13
  6 mod 13   7 mod 13   8 mod 13      1 mod 13   8 mod 13   3 mod 13
  0 mod 13   6 mod 13  12 mod 13      3 mod 13  12 mod 13   9 mod 13

That's a bit verbose. We can tweak `Base.show` a bit to make it less repetetive:

In [36]:
Base.show{n}(io::IO, k::ModInt{n}) =
    print(io, get(io, :compact, false) ? k.k : "$(k.k) mod $n")

In [37]:
A

10×10 Array{ModInt{13},2}:
  5   0  10   2   2   9   0  11   4   1
  5   4   1   4   1   9   7   5   0  12
  3   5   6   8   7  10   8   6   1   6
  3   6   8  12   0  11   3   8  12  12
  9  12   9   6   2  12   0   9   9   3
 11   2   5  12   6   0   4  11   8   2
  3   1  12   5  12  12   3   5   5   4
 12   2   9   9  10   2  11   3   6   2
  6   7   8   9   8   7   3   1   8   3
  0   6  12   4   2   7   4   3  12   9

Now let's play around...

In [38]:
A + A

10×10 Array{ModInt{13},2}:
 10   0   7   4   4   5  0   9   8   2
 10   8   2   8   2   5  1  10   0  11
  6  10  12   3   1   7  3  12   2  12
  6  12   3  11   0   9  6   3  11  11
  5  11   5  12   4  11  0   5   5   6
  9   4  10  11  12   0  8   9   3   4
  6   2  11  10  11  11  6  10  10   8
 11   4   5   5   7   4  9   6  12   4
 12   1   3   5   3   1  6   2   3   6
  0  12  11   8   4   1  8   6  11   5

In [39]:
2A

10×10 Array{ModInt{13},2}:
 10   0   7   4   4   5  0   9   8   2
 10   8   2   8   2   5  1  10   0  11
  6  10  12   3   1   7  3  12   2  12
  6  12   3  11   0   9  6   3  11  11
  5  11   5  12   4  11  0   5   5   6
  9   4  10  11  12   0  8   9   3   4
  6   2  11  10  11  11  6  10  10   8
 11   4   5   5   7   4  9   6  12   4
 12   1   3   5   3   1  6   2   3   6
  0  12  11   8   4   1  8   6  11   5

In [40]:
A*A

10×10 Array{ModInt{13},2}:
  9   4   7   9   9  1  12   2   7   0
  2   8   3  12   1  0   0   8   8   1
  6   5  10   4   4  6   4   5  10  12
  0   8  11   2   6  6   3  12   9   6
  7   3   3   7  12  3   9   1   3   4
 11  11   7   7  11  4   8   1   5   8
  7   9  12   9   9  9   1   8   5   7
  3   3   7  11  10  5   5   1   0   6
  9  12  10  10   2  0   3   2  12  11
  7   8   0   0   3  1   9  11  12   0

In [45]:
map(a->a.k, A)^2

10×10 Array{Int64,2}:
 334  160  332  373  282  248  259  288  254  156
 249  164  419  324  235  325  208  307  359  274
 357  265  478  459  303  461  264  408  413  337
 403  333  583  613  370  513  367  415  555  461
 462  276  471  540  337  471  360  456  393  381
 362  271  527  449  297  511  255  378  421  307
 410  334  480  503  321  464  261  437  421  280
 341  302  579  440  335  629  213  456  416  318
 334  311  439  452  249  507  224  405  389  349
 293  294  442  455  328  443  295  297  376  351

In [46]:
map(a->(a%13), ans)

10×10 Array{Int64,2}:
  9   4   7   9   9  1  12   2   7   0
  2   8   3  12   1  0   0   8   8   1
  6   5  10   4   4  6   4   5  10  12
  0   8  11   2   6  6   3  12   9   6
  7   3   3   7  12  3   9   1   3   4
 11  11   7   7  11  4   8   1   5   8
  7   9  12   9   9  9   1   8   5   7
  3   3   7  11  10  5   5   1   0   6
  9  12  10  10   2  0   3   2  12  11
  7   8   0   0   3  1   9  11  12   0

In [47]:
A^2

10×10 Array{ModInt{13},2}:
  9   4   7   9   9  1  12   2   7   0
  2   8   3  12   1  0   0   8   8   1
  6   5  10   4   4  6   4   5  10  12
  0   8  11   2   6  6   3  12   9   6
  7   3   3   7  12  3   9   1   3   4
 11  11   7   7  11  4   8   1   5   8
  7   9  12   9   9  9   1   8   5   7
  3   3   7  11  10  5   5   1   0   6
  9  12  10  10   2  0   3   2  12  11
  7   8   0   0   3  1   9  11  12   0

In [48]:
A^100

10×10 Array{ModInt{13},2}:
 3  4   2   5   1   1  0   8  11   0
 1  8   8   8  11   0  7   2  11   9
 9  9  12   1  11   4  2   0  12   9
 1  4   4   1   9   2  9  10   2  12
 7  8   3   5  12   5  4  11   4   3
 2  0   7   9   1  10  4  10   5   3
 8  4   6   7   7  10  0   2  10   6
 7  7  11   5   5   5  8   9  11  11
 2  4  11  11   2   3  5   2  11  11
 1  1   5   8  10   1  7  11  12   8

In [50]:
A^1000000

10×10 Array{ModInt{13},2}:
  2  12   3   1  6  11   7   1   2   0
  9   2  11  10  4   5   8   4   1   2
  3   9   0  11  9   4  11  12   9   8
  8   4   8   7  1   9  11   9   4   1
  0   2  12   7  0  10   1   3   7  10
  4   9   9   7  7  11   5   1   5   2
 12  10   3   9  3   2   1   4  11  11
  4   6   6   1  6   8   7   7   8   1
  6   3   4   1  8   3  10   6   5   5
 12   8   6   6  7   7   5  12  12  10

In [51]:
@which A^1000000

^(A::AbstractArray{T,2}, p::Integer) where T in Base.LinAlg at linalg/dense.jl:332

In [52]:
sizeof(A)

800

In [53]:
sizeof(A) ÷ prod(size(A))

8

In [54]:
dump(a)

ModInt{13}
  k: Int64 5


In [55]:
a.k

5

In [56]:
a.k = 6

LoadError: [91mtype ModInt is immutable[39m

In [57]:
v = [1,2,3]

3-element Array{Int64,1}:
 1
 2
 3

In [58]:
w = [1,2,3]

3-element Array{Int64,1}:
 1
 2
 3

In [59]:
v == w

true

In [60]:
v[1] *= -1

-1

In [61]:
v

3-element Array{Int64,1}:
 -1
  2
  3

In [62]:
w

3-element Array{Int64,1}:
 1
 2
 3

In [63]:
mutable struct MudInt{n} <: Integer
    k::Int
    (MudInt{n}(k) where {n}) = new(mod(k,n))
end

In [64]:
x = MudInt{7}(123)

MudInt{7}(4)

In [65]:
x.k

4

In [66]:
x.k = 5

5

In [67]:
x

MudInt{7}(5)

In [68]:
X = map(MudInt{7}, rand(Int, 5, 5))

5×5 Array{MudInt{7},2}:
 MudInt{7}(5)  MudInt{7}(3)  MudInt{7}(1)  MudInt{7}(1)  MudInt{7}(5)
 MudInt{7}(4)  MudInt{7}(5)  MudInt{7}(6)  MudInt{7}(1)  MudInt{7}(1)
 MudInt{7}(6)  MudInt{7}(0)  MudInt{7}(5)  MudInt{7}(5)  MudInt{7}(3)
 MudInt{7}(6)  MudInt{7}(5)  MudInt{7}(5)  MudInt{7}(0)  MudInt{7}(0)
 MudInt{7}(4)  MudInt{7}(5)  MudInt{7}(5)  MudInt{7}(0)  MudInt{7}(3)

In [70]:
x = X[1]

MudInt{7}(5)

In [71]:
x.k = 7

7

In [72]:
x

MudInt{7}(7)

In [73]:
X

5×5 Array{MudInt{7},2}:
 MudInt{7}(7)  MudInt{7}(3)  MudInt{7}(1)  MudInt{7}(1)  MudInt{7}(5)
 MudInt{7}(4)  MudInt{7}(5)  MudInt{7}(6)  MudInt{7}(1)  MudInt{7}(1)
 MudInt{7}(6)  MudInt{7}(0)  MudInt{7}(5)  MudInt{7}(5)  MudInt{7}(3)
 MudInt{7}(6)  MudInt{7}(5)  MudInt{7}(5)  MudInt{7}(0)  MudInt{7}(0)
 MudInt{7}(4)  MudInt{7}(5)  MudInt{7}(5)  MudInt{7}(0)  MudInt{7}(3)

In [80]:
y = deepcopy(x)

MudInt{7}(11)

In [81]:
y.k = 3

3

In [82]:
y

MudInt{7}(3)

In [83]:
X

5×5 Array{MudInt{7},2}:
 MudInt{7}(11)  MudInt{7}(3)  MudInt{7}(1)  MudInt{7}(1)  MudInt{7}(5)
  MudInt{7}(4)  MudInt{7}(5)  MudInt{7}(6)  MudInt{7}(1)  MudInt{7}(1)
  MudInt{7}(6)  MudInt{7}(0)  MudInt{7}(5)  MudInt{7}(5)  MudInt{7}(3)
  MudInt{7}(6)  MudInt{7}(5)  MudInt{7}(5)  MudInt{7}(0)  MudInt{7}(0)
  MudInt{7}(4)  MudInt{7}(5)  MudInt{7}(5)  MudInt{7}(0)  MudInt{7}(3)

In [79]:
x

MudInt{7}(11)